<a href="https://colab.research.google.com/github/tourihasi/Openstudio/blob/main/%E2%93%AA-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ================================
# gbXMLを90°回転する（Google Colab用）
# - エクスプローラからgbXMLファイルを選択
# - Z軸まわりに+90°（既定）で回転
# - PlanarGeometryのCartesianPointを回転し、
#   Azimuthがあれば+90°補正（0-360正規化）
# - 出力ファイルを自動ダウンロード
# ================================

!pip -q install lxml

from lxml import etree
from google.colab import files
import io, math, sys

# ---- ①設定（必要に応じて変更） ----
ROTATE_DEG = 90.0      # 回転角（度）
AXIS = "Z"             # 回転軸: "X" / "Y" / "Z"
CENTER_MODE = "centroid"  # 回転中心: "centroid"（重心） or "origin"（原点）
DECIMALS = 6           # 座標の小数桁（丸め）
ADJUST_AZIMUTH = True  # Azimuthも補正するか（Z軸回りのとき推奨）

# ---- ②ファイル選択（エクスプローラ） ----
print("処理したいgbXMLファイルを選択してください...")
uploaded = files.upload()
if not uploaded:
    print("ファイルが選択されませんでした。中止します。")
    raise SystemExit

in_name = next(iter(uploaded.keys()))
in_bytes = uploaded[in_name]
print(f"読み込み中: {in_name}")

# ---- ③パーサ設定（名前空間対応） ----
parser = etree.XMLParser(remove_blank_text=False)
tree = etree.parse(io.BytesIO(in_bytes), parser)
root = tree.getroot()

# ④定名前空間を取得（gbXMLは既定NSが多い）
def ns_prefix():
    # root.tag は "{ns}gbXML" の形になっている可能性
    if root.tag.startswith("{"):
        ns = root.tag.split("}")[0][1:]
        return "{%s}" % ns
    # nsmap から既定NSを取る
    ns = root.nsmap.get(None, '')
    return "{%s}" % ns if ns else ""

NS = ns_prefix()

# ---- ⑤回転行列の準備 ----
theta = math.radians(ROTATE_DEG)
cos_t, sin_t = math.cos(theta), math.sin(theta)

def rot_point(x, y, z):
    if AXIS.upper() == "Z":
        return (x*cos_t - y*sin_t, x*sin_t + y*cos_t, z)
    elif AXIS.upper() == "Y":
        # Y軸回り: (x,z)を回転
        return (x*cos_t + z*sin_t, y, -x*sin_t + z*cos_t)
    elif AXIS.upper() == "X":
        # X軸回り: (y,z)を回転
        return (x, y*cos_t - z*sin_t, y*sin_t + z*cos_t)
    else:
        raise ValueError("AXISは 'X' 'Y' 'Z' のいずれかにしてください。")

# ---- ⑥すべてのCartesianPointを収集 ----
cart_pts = root.findall(f".//{NS}CartesianPoint")
if not cart_pts:
    print("CartesianPointが見つかりませんでした。gbXMLの構造をご確認ください。")
    raise SystemExit

# ---- ⑦回転中心（重心 or 原点）を決定 ----
def read_xyz(pt):
    coords = pt.findall(f"{NS}Coordinate")
    # 2D/3D混在に備えて足りない軸は0とみなす
    vals = [float(c.text) for c in coords[:3]] + [0.0]*(3-len(coords))
    return vals[0], vals[1], vals[2]

if CENTER_MODE.lower() == "centroid":
    sx = sy = sz = n = 0.0
    for pt in cart_pts:
        x, y, z = read_xyz(pt)
        sx += x; sy += y; sz += z; n += 1
    cx, cy, cz = sx/n, sy/n, sz/n
else:
    cx = cy = cz = 0.0

# ---- ⑧座標を回転し上書き ----
def rotate_and_write(pt):
    coords = pt.findall(f"{NS}Coordinate")
    # 足りない軸は0で補う
    xyz = [float(c.text) for c in coords[:3]] + [0.0]*(3-len(coords))
    x0, y0, z0 = xyz[0] - cx, xyz[1] - cy, xyz[2] - cz
    xr, yr, zr = rot_point(x0, y0, z0)
    xr, yr, zr = xr + cx, yr + cy, zr + cz
    # 座標を書き戻す（存在する要素分のみ）
    if len(coords) >= 1: coords[0].text = f"{xr:.{DECIMALS}f}"
    if len(coords) >= 2: coords[1].text = f"{yr:.{DECIMALS}f}"
    if len(coords) >= 3: coords[2].text = f"{zr:.{DECIMALS}f}"

for pt in cart_pts:
    rotate_and_write(pt)

# ---- ⑨Azimuthの補正（Z軸回りのとき有効） ----
if ADJUST_AZIMUTH and AXIS.upper() == "Z" and abs(ROTATE_DEG) % 360.0 != 0.0:
    az_nodes = root.findall(f".//{NS}Azimuth")
    for az in az_nodes:
        try:
            val = float(az.text)
            val = (val + ROTATE_DEG) % 360.0
            az.text = f"{val:.3f}"
        except:
            pass

# ---- ⑩出力＆ダウンロード ----
suffix = f"_rot{AXIS.upper()}_{int(ROTATE_DEG)}"
out_name = in_name.rsplit(".", 1)[0] + f"{suffix}.gbxml"

# pretty_printは要件次第。XML宣言とエンコーディングは付与
tree.write(out_name, encoding="utf-8", xml_declaration=True, pretty_print=True)

print(f"変換完了: {out_name}")
files.download(out_name)
